# MLOps Zoomcamp Homework 1 Notebook

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
train_df = pd.read_parquet("fhv_tripdata_2021-01.parquet")

### Let's calculate the trip duration by subtracting the two datetime values and dividing by 60 to get the result in hours.

In [3]:
print(f"There are {train_df.shape[0]} number of records in the training dataset.")

There are 1154112 number of records in the training dataset.


### Taking the difference between pickup and dropoff time and converting it from seconds to minutes.

In [4]:
train_df['duration'] = train_df.dropOff_datetime - train_df.pickup_datetime
train_df.duration = train_df.duration.apply(lambda trip_duration: trip_duration.total_seconds() / 60)

In [5]:
print(f"The average trip duration is:", "{:.2f}".format(train_df.duration.mean()), "minutes")

The average trip duration is: 19.17 minutes


In [6]:
train_df = train_df[(train_df.duration >= 1) & (train_df.duration <= 60)]

In [7]:
print(f"The number of records dropped after filtering is: {1154112 - train_df.shape[0]}")

The number of records dropped after filtering is: 44286


### Replacing the NaN values in the Pickup and Dropof location ID columns with -1 and calculating the percentages of missing values (-1s)

In [8]:
train_df.PUlocationID = train_df.PUlocationID.fillna(-1)
train_df.DOlocationID = train_df.DOlocationID.fillna(-1)

In [9]:
print("% of missing values in the PUlocationID columns is: ", "{:.2f}".format(train_df["PUlocationID"].value_counts()[-1] / train_df.PUlocationID.count()))
print("% of missing values in the DOlocationID columns is: ", "{:.2f}".format(train_df["DOlocationID"].value_counts()[-1] / train_df.DOlocationID.count()))

% of missing values in the PUlocationID columns is:  0.84
% of missing values in the DOlocationID columns is:  0.13


### OneHot Encoding The Pickup and Dropoff Location ID's

We'll use sklearn's DictVectorizer which does binary one-hot encoding.

In [10]:
dict_vec = DictVectorizer()
features_to_dict = ["PUlocationID", "DOlocationID",]
train_df_columns = train_df[features_to_dict].to_dict(orient = "records")
X_train = dict_vec.fit_transform(train_df_columns)

In [12]:
print(f"The number of columns in the matrix after one-hot encoding is: {X_train.shape[1]}")

The number of columns in the matrix after one-hot encoding is: 2


### Now let's train the Linear Regression model with the one-hot encoded matrix of pickup and dropoff location ID's

In [14]:
y_train = train_df["duration"].values

In [38]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
y_pred = lin_reg.predict(X_train)

mean_squared_error(y_train, y_pred, squared = False)

11.415432830521663

In [17]:
val_df = pd.read_parquet("fhv_tripdata_2021-02.parquet")

In [27]:
val_df['duration'] = val_df.dropOff_datetime - val_df.pickup_datetime
val_df.duration = val_df.duration.apply(lambda trip_duration: trip_duration.total_seconds() / 60)

In [28]:
val_df = val_df[(val_df.duration >= 1) & (val_df.duration <= 60)]

In [22]:
val_df.PUlocationID = val_df.PUlocationID.fillna(-1)
val_df.DOlocationID = val_df.DOlocationID.fillna(-1)

In [25]:
val_df_columns = val_df[features_to_dict].to_dict(orient = "records")
X_val = dict_vec.fit_transform(val_df_columns)

In [29]:
y_val = val_df['duration']

In [35]:
y_val.shape

(990113,)

In [36]:
y_pred.shape

(1037692,)

In [37]:
X_val.shape

(1037692, 2)

In [39]:
y_pred = lin_reg.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

ValueError: Found input variables with inconsistent numbers of samples: [990113, 1037692]

The lengths of y_val and y_pred are not matching.